## Homework Grading
The goal of this is to develop a robust homework grading system which combines Google Colab + Jupyter Notebooks + GitHub Classroom. 

For this example, we are going to download a zip file with a sample assignments.  However, if you end up grading from colab, you may consider directly mounting your google drive with the code below.  


In [ ]:
#This is used when grading from Colab. 
#from google.colab import drive
#drive.mount('/content/drive')
#!ls -al /content/drive/'My Drive'/autograding


In [ ]:
# This installs the forked version of Gopher Grader. 
#!pip install git+https://github.com/CarmeLabs/Gofer-Grader 

In [ ]:
#set to what you want to grade
course = 'introml'
assignment = 'assignment1' 
  #use sample for a demo assignment or assignment name.

### Setup for Grading

Make sure you have installed the dependencies in the `requirements.txt` file. 

`pip install -r requirements.txt`

Make sure you have all of the required files.

1.  Download assignment from github and put it in the `assignments/<course>/<assignment>
2. Update the `/config/<course>.ini` file with the appropriate assignment file. 
3. Verify that the appropriate tests and data are in the associated `/tests/<course>/assignment` folder. 
4. Verify that the roster is setup in `roster/course.xlsx`. 
5. Verify other configurations in  `/config/<course>.ini` are correct.  

If you want to delete the tmp grading folder set `delete_tmp` to `True`. 

In [ ]:
delete_tmp = False

#TBD. Do a check which prechecks for 
#the existance of all required files as stated above. 
#Give option to delete the /tmp folder. 

### Set and Load the Configuration
We save configuration in a local file. 

In [ ]:
#Set configuration
from pathlib import Path
import configparser, sys, os, importlib
import pandas as pd
 
cwd_dir = Path.cwd() #For running locally
base_dir = cwd_dir.parent #For running locally
#base_dir='/content/drive/My Drive/autograding'
course_file = course+'.ini'  #config file 
config_file = base_dir / 'config' / course_file
modules_path = base_dir / 'modules' 
sys.path.append('../modules') # Not sure why appending the mudles path didn't work. 

#Load the autograding library
import autograde as ag
importlib.reload(ag)

#read in the configuration
config = configparser.ConfigParser()
config.read(config_file)
cf = ag.set_config(course, assignment, base_dir, config)
cf
#cf

## Grade Students

This `ag.grade` function is used to grade students. This will copy from the directory found in the config settings to the /tmp directory. 

`cf`   # This is the config. 

`grade` # The number of students to grade.  Usually I start with a few. 

`cleanup` # This will delete the temp directory when done.

`regrade` # This will regrade students or no. 

`submissions` # This is a list of students to grade and can be used for selective regrading. 

This will generate output include:

This function generates grading output in:
`/output/<course>/<assignment>/` that includes a status, json output, and the executed notebooks. 




In [ ]:
#This generates grades as JSON 
importlib.reload(ag) #Reload model if working on it so that it can be reloaded.
#Leave submissions blank to grade all submissions. 
result = ag.grade(cf, grade =100, cleanup=False, regrade=True, submissions=[])    
result

## Aggregate Values
No you have graded all the students there is an output file in the /tmp directory with their grades.  The `generate_mangrade` command below will aggregate the values to a CSV file.  

This will also match the githubID with the person using the roster file.  



In [ ]:
#Aggregate JSON files for manual grading. 
importlib.reload(ag)
mangrade_df=ag.generate_mangrade(cf)
mangrade_df

# Matching Github ID to USERNAME.
Currently this is setup to upload to Blackboard.

It matches the github name with the student name in the roster file. 

In [ ]:
#Update Manual Grading to include email, etc. Make sure only run 1 time. 
github_df=pd.read_excel(cf['roster_path'], sheet_name='github')
mangrade_df_updated=mangrade_df.merge(github_df, left_on='github_id', right_on='github_id', how = 'left')
mangrade_df_updated=mangrade_df_updated.sort_values(by=['lastname_r','firstname_r'])
mangrade_df_updated.to_excel(cf['mangrade_output_path'], sheet_name='mangrade', index=False)



## Grading 
Now go ahead and manually grade all manual questions.

Create a column total which includes `autograde_total` plus any manual grading. 



## Generating Blackboard 

This will generate a blackboard upload dataframe and message variables. 

In [ ]:
#Read in the updated grading file. 
mangrade_df=pd.read_excel(cf['mangrade_output_path'])
message_complete = """Your submission was successfully received and graded.  The attached result is the score 
received for each question as well as the output if an error was obtained.  
<br>

"""

message_incomplete = """If you get this message see the instructor. 
"""
github_df=pd.read_excel(cf['roster_path'], sheet_name='github')
blackboard_df=pd.read_excel(cf['roster_path'], sheet_name='blackboard')
blackboard_df.rename(columns={cf['blackboard_rename_col']: cf['blackboard_total_col']}, inplace=True)
blackboard_df

### Output Grade to Blackboard CSV
This will generate the output `blackboard.csv` file which can be updated 

In [ ]:
#counter
complete = 0
incomplete = 0
print(cf['mangrade_match_col'],cf['blackboard_match_col'] )
for index, row in blackboard_df.iterrows():
    github_id=None
    github_id=github_df.loc[github_df['username_r'] == row['Username'],'github_id'].values
    if (len(github_id))>0:
        github_id=github_id[0]
        match_row=mangrade_df.loc[mangrade_df['github_id'] == github_id,]
        if len(match_row)>0:
            text=""
            for x in match_row.columns:
                text=text+x+": "+str( match_row[x].ravel()[0])+"<br>"
            blackboard_df.loc[index,cf['blackboard_total_col']]=match_row['total'].ravel()[0]
            blackboard_df.loc[index,'Feedback Format']='HTML'
            blackboard_df.loc[index,'Feedback to Learner']=message_complete+text
            complete =complete+1
        else:
            incomplete =incomplete+1
            blackboard_df.loc[index,cf['blackboard_total_col']]=0
            blackboard_df.loc[index,'Feedback to Learner']=message_incomplete
            incomplete=incomplete+1
    else:
        incomplete =incomplete+1
        blackboard_df.loc[index,cf['blackboard_total_col']]=0
        blackboard_df.loc[index,'Feedback to Learner']=message_incomplete
        incomplete=incomplete+1
    

    #blackboard_df.loc[index,blackboard_total_col]=mangrade_df.loc[mangrade_df[mangrade_match_col] == row[blackboard_match_col],'total'].ravel()[0]
blackboard_df.to_csv(cf['blackboard_output_path'], index = False)
print("complete:", complete,"\nIncomplete:",incomplete,"\nTotal:",complete+incomplete)
blackboard_df


## Upload the Blackboard File 

And you are done. 